<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/nvidia_neo4j_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet langchain-nvidia-ai-endpoints langchain-community neo4j langchain-core nemoguardrails

In [2]:
import os

from typing import Optional, Type, List, Dict, Tuple, Any

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import AIMessage, HumanMessage
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


In [3]:
os.environ["NVIDIA_API_KEY"] = "nvapi-"

In [4]:
llm = ChatNVIDIA(model="meta/llama-3.1-70b-instruct")

In [5]:
result = llm.invoke("How to use LLMs in combination with Graph Databases? Be concise!")
print(result.content)

Here's a concise guide on using Large Language Models (LLMs) with Graph Databases:

**Why combine LLMs with Graph Databases?**

* Enhance querying and reasoning capabilities
* Inject domain knowledge and contextual understanding
* Leverage graph structures for entity recognition and disambiguation

**Possible combinations:**

1. **Entity recognition and disambiguation**: Use LLMs to identify entities in text and match them to nodes in the graph database.
2. **Question answering**: Utilize LLMs to analyze natural language queries and convert them into graph database queries.
3. **Graph node classification**: Train LLMs on graph node attributes and relationships to predict node labels or types.
4. **Graph-based text generation**: Leverage graph structures to guide LLM-based text generation, incorporating domain knowledge and relationships.
5. **Knowledge graph completion**: Use LLMs to infer missing relationships and entities in the graph database.

**Tools and architectures:**

* Graph 

In [6]:
os.environ["NEO4J_URI"] = "bolt://18.206.157.187:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "elevation-reservist-thousands"

graph = Neo4jGraph(refresh_schema=False)

In [7]:
graph.query(
    "CREATE FULLTEXT INDEX drug IF NOT EXISTS FOR (d:Drug) ON EACH [d.name];"
)
graph.query(
    "CREATE FULLTEXT INDEX manufacturer IF NOT EXISTS FOR (d:Manufacturer) ON EACH [d.manufacturerName];"
)

[]

In [8]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2) to each word, then combines them using the AND
    operator. Useful for mapping movies and people from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


candidate_query = """
CALL db.index.fulltext.queryNodes($index, $fulltextQuery, {limit: $limit})
YIELD node
RETURN coalesce(node.manufacturerName, node.name) AS candidate,
       labels(node)[0] AS label
"""


def get_candidates(input: str, type: str, limit: int = 3) -> List[Dict[str, str]]:
    """
    Retrieve a list of candidate entities from database based on the input string.

    This function queries the Neo4j database using a full-text search. It takes the
    input string, generates a full-text query, and executes this query against the
    specified index in the database. The function returns a list of candidates
    matching the query, with each candidate being a dictionary containing their name
    (or title) and label (either 'Person' or 'Movie').
    """
    ft_query = generate_full_text_query(input)
    candidates = graph.query(
        candidate_query, {"fulltextQuery": ft_query, "index": type, "limit": limit}
    )
    return candidates

In [9]:
get_candidates("Acadia", "manufacturer")

[{'candidate': 'ACADIA PHARMACEUTICALS', 'label': 'Manufacturer'}]

In [10]:
get_candidates("voriconazol", "drug")

[{'candidate': 'VORICONAZOLE', 'label': 'Drug'}]

In [11]:



def get_side_effects(
    drug: Optional[str] = None,
    min_age: Optional[int] = None,
    max_age: Optional[int] = None,
    manufacturer: Optional[str] = None,
) -> str:
    """Get the side effects of a drug."""
    params = {}
    filters = []
    side_effects_base_query = """
    MATCH (c:Case)-[:HAS_REACTION]->(r:Reaction), (c)-[:IS_PRIMARY_SUSPECT]->(d:Drug)
    """
    if drug or min_age or max_age or manufacturer:
        side_effects_base_query += " WHERE "
    if drug:
        candidate_drugs = [el["candidate"] for el in get_candidates(drug, "drug")]
        if not candidate_drugs:
            return "The mentioned drug was not found"
        filters.append("d.name IN $drugs")
        params["drugs"] = candidate_drugs
    if min_age:
        filters.append("c.age > $min_age ")
        params["min_age"] = min_age
    if max_age:
        filters.append("c.age < $max_age ")
        params["max_age"] = max_age
    if manufacturer:
        candidate_manufacturers = [
            el["candidate"] for el in get_candidates(manufacturer, "manufacturer")
        ]
        if not candidate_manufacturers:
            return "The mentioned manufacturer was not found"
        filters.append(
            "EXISTS {(c)<-[:REGISTERED]-(:Manufacturer {manufacturerName: $manufacturer})}"
        )
        params["manufacturer"] = candidate_manufacturers[0]
    if filters:
        side_effects_base_query += " AND ".join(filters)
    side_effects_base_query += """
    RETURN d.name AS drug, r.description AS side_effect, count(*) AS count
    ORDER BY count DESC
    LIMIT 10
    """
    print(side_effects_base_query)
    print(params)
    data = graph.query(side_effects_base_query, params=params)
    return data

In [12]:
get_side_effects(manufacturer="acadia")


    MATCH (c:Case)-[:HAS_REACTION]->(r:Reaction), (c)-[:IS_PRIMARY_SUSPECT]->(d:Drug)
     WHERE EXISTS {(c)<-[:REGISTERED]-(:Manufacturer {manufacturerName: $manufacturer})}
    RETURN d.name AS drug, r.description AS side_effect, count(*) AS count
    ORDER BY count DESC
    LIMIT 10
    
{'manufacturer': 'ACADIA PHARMACEUTICALS'}


[{'drug': 'NUPLAZID', 'side_effect': 'Hallucination', 'count': 13},
 {'drug': 'NUPLAZID', 'side_effect': 'Confusional state', 'count': 7},
 {'drug': 'NUPLAZID', 'side_effect': 'Fall', 'count': 6},
 {'drug': 'NUPLAZID', 'side_effect': 'Delusion', 'count': 5},
 {'drug': 'NUPLAZID', 'side_effect': 'Gait disturbance', 'count': 5},
 {'drug': 'NUPLAZID', 'side_effect': 'Fatigue', 'count': 4},
 {'drug': 'NUPLAZID', 'side_effect': 'Abnormal behaviour', 'count': 3},
 {'drug': 'NUPLAZID',
  'side_effect': 'Product dose omission issue',
  'count': 3},
 {'drug': 'NUPLAZID', 'side_effect': 'Agitation', 'count': 3},
 {'drug': 'NUPLAZID', 'side_effect': 'Death', 'count': 3}]

In [13]:
get_side_effects(drug="aspirin")


    MATCH (c:Case)-[:HAS_REACTION]->(r:Reaction), (c)-[:IS_PRIMARY_SUSPECT]->(d:Drug)
     WHERE d.name IN $drugs
    RETURN d.name AS drug, r.description AS side_effect, count(*) AS count
    ORDER BY count DESC
    LIMIT 10
    
{'drugs': ['ASPIRINE', 'ASPRIN', 'ASPIRIN']}


[]

In [26]:
from langchain_core.pydantic_v1 import Field
from langchain_core.tools import tool


@tool
def get_side_effects(
    drug: Optional[str] = Field(description="disease mentioned in the question. Return None if no mentioned."),
    min_age: Optional[int] = Field(description="Minimum age of the patient. Return None if no mentioned."),
    max_age: Optional[int] = Field(description="Maximum age of the patient. Return None if no mentioned."),
    manufacturer: Optional[str] = Field(description="manufacturer of the drug. Return None if no mentioned.")
):
    """Useful for when you need to find common side effects."""
    params = {}
    filters = []
    side_effects_base_query = """
    MATCH (c:Case)-[:HAS_REACTION]->(r:Reaction), (c)-[:IS_PRIMARY_SUSPECT]->(d:Drug)
    """
    if drug:
        side_effects_base_query += " WHERE "
    if drug:
        candidate_drugs = [el["candidate"] for el in get_candidates(drug, "drug")]
        if not candidate_drugs:
            return "The mentioned drug was not found"
        filters.append("d.name IN $drugs")
        params["drugs"] = candidate_drugs

    if min_age:
        filters.append("c.age > $min_age ")
        params["min_age"] = min_age
    if max_age:
        filters.append("c.age < $max_age ")
        params["max_age"] = max_age
    if manufacturer:
        candidate_manufacturers = [
            el["candidate"] for el in get_candidates(manufacturer, "manufacturer")
        ]
        if not candidate_manufacturers:
            return "The mentioned manufacturer was not found"
        filters.append(
            "EXISTS {(c)<-[:REGISTERED]-(:Manufacturer {manufacturerName: $manufacturer})}"
        )
        params["manufacturer"] = candidate_manufacturers[0]

    if filters:
        side_effects_base_query += " AND ".join(filters)
    side_effects_base_query += """
    RETURN d.name AS drug, r.description AS side_effect, count(*) AS count
    ORDER BY count DESC
    LIMIT 10
    """
    print(side_effects_base_query)
    print(params)
    data = graph.query(side_effects_base_query, params=params)
    return data

In [27]:
tools = [get_side_effects]
llm_with_tools = llm.bind_tools(tools=tools)
response = llm_with_tools.invoke("What is the most common side effect when using volazopile?")
print(response)
print(response.tool_calls)

content='' additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-899e69df1b4e4c319b95065606366e4f', 'type': 'function', 'function': {'name': 'get_side_effects', 'arguments': '{"drug": "volazopile"}'}}]} response_metadata={'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'chatcmpl-tool-899e69df1b4e4c319b95065606366e4f', 'type': 'function', 'function': {'name': 'get_side_effects', 'arguments': '{"drug": "volazopile"}'}}], 'token_usage': {'prompt_tokens': 315, 'total_tokens': 343, 'completion_tokens': 28}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.1-70b-instruct'} id='run-ea03329f-67c6-4037-9087-10fcd1656727-0' tool_calls=[{'name': 'get_side_effects', 'args': {'drug': 'volazopile'}, 'id': 'chatcmpl-tool-899e69df1b4e4c319b95065606366e4f', 'type': 'tool_call'}] role='assistant'
[{'name': 'get_side_effects', 'args': {'drug': 'volazopile'}, 'id': 'chatcmpl-tool-899e69df1b4e4c319b95065606366e4f', 'type': 'tool_call'}]


In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that finds information about common side effects. "
            "If tools require follow up questions, "
            "make sure to ask the user for clarification. Make sure to include any "
            "available options that need to be clarified in the follow up questions "
            "Do only the things the user specifically requested. ",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [29]:
def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


agent = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"])
        if x.get("chat_history")
        else [],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)


# Add typing for input
class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


class Output(BaseModel):
    output: Any


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_types(
    input_type=AgentInput, output_type=Output
)

In [30]:
agent_executor.invoke({"input": "What is the most common side effect when using volazopile?"})



> Entering new AgentExecutor chain...


> Finished chain.


{'input': 'What is the most common side effect when using volazopile?',
 'output': ''}